# Camera geometry play

Craig Lage - 15-Jan-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import lsst.afw.cameraGeom.utils as camGeomUtils
from lsst.afw.cameraGeom import DetectorType
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.geom import Point2D, radians

In [ ]:
butler = Butler('/repo/main', collections=["LSSTCam/raw/all", "LSSTCam/calib"])
instrument = 'LSSTCam'
camera = butler.get('camera', instrument=instrument)

# This gives you the location of a given CCD in the focal plane, in mm.

## With this, you could take a given pixel within a given CCD and find the location on the focal plane in mm.

In [ ]:
ccd = camera['R01_S00']
corners = ccd.getCorners(camGeomUtils.FOCAL_PLANE)
for corner in corners:
    print(corner) # These are in mm, with (0,0) in the camera center

In [ ]:
import lsst.afw.cameraGeom.utils as camGeomUtils
from lsst.daf.butler import Butler
butler = Butler('/repo/main', collections=["LSSTCam/raw/all", "LSSTCam/calib"])
instrument = 'LSSTCam'
camera = butler.get('camera', instrument=instrument)
ccd = camera['R01_S00']
corners = ccd.getCorners(camGeomUtils.FOCAL_PLANE)
for corner in corners:
    print(corner) # These are in mm, with (0,0) in the camera center

# The function below is copied from lsst.afw.cameraGeom.utils, but I had to correct an error.  This correction has been pushed to the main afw code.

In [ ]:
def plotFocalPlane(camera, fieldSizeDeg_x=0, fieldSizeDeg_y=None, dx=0.1, dy=0.1, figsize=(10., 10.),
                   useIds=False, showFig=True, savePath=None):
    """Make a plot of the focal plane along with a set points that sample
    the field of view.

    Parameters
    ----------
    camera : `lsst.afw.cameraGeom.Camera`
        A camera object
    fieldSizeDeg_x : `float`
        Amount of the field to sample in x in degrees
    fieldSizeDeg_y : `float` or `None`
        Amount of the field to sample in y in degrees
    dx : `float`
        Spacing of sample points in x in degrees
    dy : `float`
        Spacing of sample points in y in degrees
    figsize : `tuple` containing two `float`
        Matplotlib style tuple indicating the size of the figure in inches
    useIds : `bool`
        Label detectors by name, not id?
    showFig : `bool`
        Display the figure on the screen?
    savePath : `str` or `None`
        If not `None`, save a copy of the figure to this name.
    """
    try:
        from matplotlib.patches import Polygon
        from matplotlib.collections import PatchCollection
        import matplotlib.pyplot as plt
    except ImportError:
        raise ImportError(
            "Can't run plotFocalPlane: matplotlib has not been set up")

    if fieldSizeDeg_x:
        if fieldSizeDeg_y is None:
            fieldSizeDeg_y = fieldSizeDeg_x

        field_gridx, field_gridy = np.meshgrid(
            np.arange(0., fieldSizeDeg_x + dx, dx) - fieldSizeDeg_x/2.,
            np.arange(0., fieldSizeDeg_y + dy, dy) - fieldSizeDeg_y/2.)
        field_gridx, field_gridy = field_gridx.flatten(), field_gridy.flatten()
    else:
        field_gridx, field_gridy = [], []

    xs = []
    ys = []
    pcolors = []

    # compute focal plane positions corresponding to field angles field_gridx, field_gridy
    posFieldAngleList = [Point2D(x*radians, y*radians)
                         for x, y in zip(field_gridx, field_gridy)]
    posFocalPlaneList = camera.transform(posFieldAngleList, camGeomUtils.FIELD_ANGLE, camGeomUtils.FOCAL_PLANE)
    for posFocalPlane in posFocalPlaneList:
        xs.append(posFocalPlane.getX())
        ys.append(posFocalPlane.getY())
        dets = camera.findDetectors(posFocalPlane, camGeomUtils.FOCAL_PLANE)
        if len(dets) > 0:
            pcolors.append('w')
        else:
            pcolors.append('k')

    colorMap = {DetectorType.SCIENCE: 'b', DetectorType.FOCUS: 'y',
                DetectorType.GUIDER: 'g', DetectorType.WAVEFRONT: 'r'}

    patches = []
    colors = []
    plt.figure(figsize=figsize)
    ax = plt.gca()
    xvals = []
    yvals = []
    for det in camera:
        corners = [(c.getX(), c.getY()) for c in det.getCorners(camGeomUtils.FOCAL_PLANE)]
        for corner in corners:
            xvals.append(corner[0])
            yvals.append(corner[1])
        colors.append(colorMap[det.getType()])
        patches.append(Polygon(corners, closed=True))
        center = det.getOrientation().getFpPosition()
        #print(det.getName(), det.getOrientation().getNQuarter())
        if det.getName() in ['R04_SW0', 'R04_SW1', 'R40_SW0', 'R40_SW1'] :
            text_rotation = 'vertical'
        else:
            text_rotation = 'horizontal'

        ax.text(center.getX(), center.getY(), det.getId() if useIds else det.getName(),
                horizontalalignment='center', rotation=text_rotation, rotation_mode='anchor', size=6)

    patchCollection = PatchCollection(patches, alpha=0.6, facecolor=colors)
    ax.add_collection(patchCollection)
    ax.scatter(xs, ys, s=10, alpha=.7, linewidths=0., c=pcolors)
    ax.set_xlim(min(xvals) - abs(0.1*min(xvals)),
                max(xvals) + abs(0.1*max(xvals)))
    ax.set_ylim(min(yvals) - abs(0.1*min(yvals)),
                max(yvals) + abs(0.1*max(yvals)))
    ax.set_xlabel('Focal Plane X (mm)')
    ax.set_ylabel('Focal Plane Y (mm)')
    if savePath is not None:
        plt.savefig(savePath)
    if showFig:
        plt.show()



In [ ]:
plotFocalPlane(camera,fieldSizeDeg_x=1.0, fieldSizeDeg_y=1.0)

In [ ]:
path = "/home/c/cslage/u/LSSTCam/images/plotFocalPlane.png"
camGeomUtils.plotFocalPlane(camera,fieldSizeDeg_x=1.0, fieldSizeDeg_y=1.0, savePath=path)


# To find the location on the sky of a given pixel, you extract the WCS (World Coordinate System).  If you extract it from the raw image, you get close.  For the best accuracy, extract the WCS from the calexp, which has been corrected using the astrometric solution of the stars in the image.

In [ ]:
butler = butlerUtils.makeDefaultButler("LSSTComCam")
instrument = 'LSSTComCam'
camera = butler.get('camera', instrument=instrument)

In [ ]:
path = "/home/c/cslage/u/ComCam/images/plotFocalPlane.png"
camGeomUtils.plotFocalPlane(camera,fieldSizeDeg_x=1.0, fieldSizeDeg_y=1.0, savePath=path)

In [ ]:
expId = 2024121100571
detector = 4
mData = butler.get('raw.metadata', detector=detector, exposure=expId, instrument=instrument)
rawExp = butler.get('raw', detector=detector, exposure=expId, instrument=instrument)
calExp = butler.get('calexp', detector=detector, visit=expId, instrument=instrument)
rWcs = rawExp.getWcs()
cWcs = calExp.getWcs()
rawSkyCenter = rWcs.getSkyOrigin()
print("raw", rWcs)
calExpSkyCenter = cWcs.getSkyOrigin()
print("calExp", cWcs)


In [ ]:
cWcs.pixelToSky(4000, 4000)

In [ ]:
cWcs.pixelToSky(0, 0)

In [ ]:
rawExp.hasWcs()

In [ ]:
calExp.getMetadata?

In [ ]:
md = calExp.getMetadata()

In [ ]:
import lsst.daf.base as dafBase
md = dafBase.PropertySet()
calExp.setMetadata(md)

In [ ]:
ref = Point2D(0,0)
camGeomUtils.makeFocalPlaneWcs(md,(0.01, 0.01), ref)

In [ ]:
cWcs.

In [ ]:
md = cWcs.getFitsMetadata()

In [ ]:
for key in md.keys():
    print(key, md[key])